## Regression

Another important problem in Machine Learning is regression, where a relationship is estimated between one independent variable $X$ and a continuous dependent variable $Y$.
The essence of regression is, therefore, to approximate a function $f(x) = y$ by a function $f^*$, given examples $(x_i, y_i)$ such that $f(x_i) = y_i$.

In LTN one can model a regression task by defining $f^*$ as a learnable function whose parameter values $\theta$ are
constrained by data. Additionally, a regression task requires a notion of *equality*. We therefore define the
predicate $Eq$ as a smooth version of the symbol $=$ in order to turn the constraint $f(x_i) = y_i$ into a smooth
optimization problem.

In this example, we explore regression using a problem from a real estate dataset with 414 examples, each described in
terms of 6 real-numbered features:
- the transaction date (converted to a float);
- the age of the house;
- the distance to the nearest station;
- the number of convenience stores in the vicinity;
- the latitude and longitude coordinates.

The model has to predict the house price given these features in input.

For this specific task, LTN uses the following language and grounding:

**Domains:**
- $samples$, denoting the houses and their features;
- $prices$, denoting the house prices.

**Variables:**
- $x$ for the samples;
- $y$ for the prices;
- $D(x) = samples$;
- $D(y) = prices$.

**Functions:**
- $f^*(x)$: the regression function that has to be learned;
- $D_{in}(f^*) = samples$;
- $D_{out}(f^*) = prices$, where $D_{out}(.)$ is a function which returns the domain of the output of a given logical
function.

**Predicates:**
- $Eq(y_1, y_2)$: a smooth equality predicate which measures how similar $y_1$ and $y_2$ are;
- $D_{in}(Eq) = prices, prices$.

**Axioms:**

- $\forall Diag(x,y) \text{ } Eq(f^*(x), y)$: the output of $f^*$ should be equal to the ground truth $y$ for each
example $x$ given in input.

Notice again the use of $Diag$: when grounding $x$ and $y$ onto sequences of values, this is done by obeying a
one-to-one correspondence between the sequences. In other words, we aggregate pairs of corresponding samples and prices,
instead of any combination thereof.


**Grounding:**
- $\mathcal{G}(samples)=\mathbb{R}^{6}$, samples are described by 6 features;
- $\mathcal{G}(prices)=\mathbb{R}$;
- $\mathcal{G}(x) \in \mathbb{R}^{m \times 6}, \mathcal{G}(y) \in \mathbb{R}^{m \times 1}$. Notice that this specification
refers to the same number $m$ of examples for $x$ and $y$ due to the above one-to-one correspondence
obtained with the use of $Diag$;
- $\mathcal{G}(\mathrm{eq}(\mathbf{u}, \mathbf{v}))=\exp \left(-\alpha \sqrt{\sum_{j}\left(u_{j}-v_{j}\right)^{2}}\right)$,
where the hyper-parameter $\alpha$ is a real number that scales how strict the smooth equality is. In this example, we use
$\alpha = 0.05$. Intuitively, the smooth equality is $\operatorname{exp}(- \alpha d(\mathbf{u}, \mathbf{v}))$, where
$d(\mathbf{u}, \mathbf{v})$ is the Euclidean distance between $\mathbf{u}$ and $\mathbf{v}$. It produces a 1 if the
distance is zero; as the distance increases, the result decreases exponentially towards 0. In our example, $\mathbf{u}$
will be a vector containing the results of $f^*$ for $j$ samples, while $\mathbf{v}$ will contain the ground truths
associated to the $j$ samples. Our objective is to maximize the truth degree of this predicate;
- $\mathcal{G}(f^*(x) \mid \theta): \operatorname{MLP}_{\theta}(x)$, where $MLP_{\theta}$
is a Multi-Layer Perceptron which ends in one neuron corresponding to a price prediction (linear activation).

### Dataset

Now, let's import the dataset.

The real estate dataset has 414 examples with 6 features each. We subdivide the dataset into 330 training examples and
84 test examples.

In [1]:
import torch
import pandas as pd

data = pd.read_csv("datasets/real-estate.csv")
data = data.sample(frac=1)  # shuffle

x = torch.tensor(data[['X1 transaction date', 'X2 house age',
                     'X3 distance to the nearest MRT station',
                     'X4 number of convenience stores', 'X5 latitude', 'X6 longitude']].to_numpy()).float()

y = torch.tensor(data[['Y house price of unit area']].to_numpy()).float()

x_train, y_train = x[:330], y[:330]
x_test, y_test = x[330:], y[330:]

### LTN setting

In order to define our knowledge base (axioms), we need to define function $f$, predicate $Eq$,
universal quantifier, and the `SatAgg` operator.

For the quantifier, we use the stable product configuration (seen in the tutorials).

For function $f$, we use a simple $MLP$ with two hidden layers.

For predicate $Eq$, we use a lambda function which implements the *grounding* defined above.

`SatAgg` is defined using the `pMeanError` aggregator.

In [2]:
import ltn

# we define function f
class MLP(torch.nn.Module):
    """
    This model returns the prediction of the price of an house given in input. The output is linear since we are applying
    the model to a regression problem.
    """
    def __init__(self, layer_sizes=(6, 8, 8, 1)):
        super(MLP, self).__init__()
        self.elu = torch.nn.ELU()
        self.linear_layers = torch.nn.ModuleList([torch.nn.Linear(layer_sizes[i - 1], layer_sizes[i])
                                                  for i in range(1, len(layer_sizes))])

    def forward(self, x):
        """
        Method which defines the forward phase of the neural network for our regression task.

        :param x: the features of the example
        :return: prediction for example x
        """
        for layer in self.linear_layers[:-1]:
            x = self.elu(layer(x))
        out = self.linear_layers[-1](x)
        return out

f = ltn.Function(MLP())

# Equality Predicate - not trainable
alpha = 0.05
Eq = ltn.Predicate(func=lambda u, v: torch.exp(
    -alpha * torch.sqrt(torch.sum(torch.square(u - v), dim=1)))
)

# we define the universal quantifier and the SatAgg operator
Forall = ltn.Quantifier(ltn.fuzzy_ops.AggregPMeanError(p=2), quantifier="f")
SatAgg = ltn.fuzzy_ops.SatAgg()

### Utils

Now, we need to define some utility classes and functions.

We define a standard PyTorch data loader, which takes as input the dataset and returns a generator of batches of data.
In particular, we need a data loader instance for training data and one for testing data.

Then, we define functions to evaluate the model performances. The model is evaluated on the test set using the following metrics:
- the satisfaction level of the knowledge base: measure the ability of LTN to satisfy the knowledge;
- the RMSE (Root Mean Squared Error): measure the quality of the predictions.

In [3]:
import numpy as np
from sklearn.metrics import mean_squared_error

# this is a standard PyTorch DataLoader to load the dataset for the training and testing of the model
class DataLoader(object):
    def __init__(self,
                 x,
                 y,
                 batch_size=1,
                 shuffle=True):
        self.x = x
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle

    def __len__(self):
        return int(np.ceil(self.x.shape[0] / self.batch_size))

    def __iter__(self):
        n = self.x.shape[0]
        idxlist = list(range(n))
        if self.shuffle:
            np.random.shuffle(idxlist)

        for _, start_idx in enumerate(range(0, n, self.batch_size)):
            end_idx = min(start_idx + self.batch_size, n)
            x = self.x[idxlist[start_idx:end_idx]]
            y = self.y[idxlist[start_idx:end_idx]]

            yield x, y

# define metrics for evaluation of the model

# it computes the overall satisfaction level on the knowledge base using the given data loader (train or test)
def compute_sat_level(loader):
    mean_sat = 0
    for x_data, y_data in loader:
        x = ltn.Variable("x", x_data)
        y = ltn.Variable("y", y_data)
        mean_sat += Forall(ltn.diag(x, y), Eq(f(x), y)).value
    mean_sat /= len(loader)
    return mean_sat

# it computes the overall RMSE between the predictions and the ground truth, using the given data loader (train or test)
def compute_rmse(loader):
    mean_rmse = 0.0
    for x, y in loader:
        predictions = f.model(x).detach().numpy()
        mean_rmse += mean_squared_error(y, predictions, squared=False)
    return mean_rmse / len(loader)

# create train and test loader
train_loader = DataLoader(x_train, y_train, 64, shuffle=True)
test_loader = DataLoader(x_test, y_test, 64, shuffle=False)

### Learning

Let us define $D$ the data set of all examples. The objective function with $\mathcal{K}=\{\forall Diag(x,y) \text{ } Eq(f^*(x), y)\}$
is given by $\operatorname{SatAgg}_{\phi \in \mathcal{K}} \mathcal{G}_{\boldsymbol{\theta}, x \leftarrow \boldsymbol{D}}(\phi)$.

In practice, the optimizer uses the following loss function:

$\boldsymbol{L}=\left(1-\underset{\phi \in \mathcal{K}}{\operatorname{SatAgg}} \mathcal{G}_{\boldsymbol{\theta}, x \leftarrow \boldsymbol{B}}(\phi)\right)$

where $B$ is a mini batch sampled from $D$.

In the following, we learn our LTN in the regression task using the satisfaction of the knowledge base as
an objective. In other words, we want to learn the parameters $\theta$ of function $f^*$ in such a way the only
axiom in the knowledge base is maximally satisfied. We train our model for 500 epochs and use the `Adam` optimizer.

In [4]:
optimizer = torch.optim.Adam(f.parameters(), lr=0.0005)

for epoch in range(500):
    train_loss = 0.0
    for batch_idx, (x_data, y_data) in enumerate(train_loader):
        optimizer.zero_grad()
        # ground the variables with current batch of data
        x = ltn.Variable("x", x_data)  # samples
        y = ltn.Variable("y", y_data)  # ground truths
        sat_agg = Forall(ltn.diag(x, y), Eq(f(x), y)).value
        loss = 1. - sat_agg
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss = train_loss / len(train_loader)

    # we print metrics every 50 epochs of training
    if epoch % 50 == 0:
        print(" epoch %d | loss %.4f | Train Sat %.3f | Test Sat %.3f | Train RMSE %.3f | Test RMSE %.3f " %
              (epoch, train_loss, compute_sat_level(train_loader), compute_sat_level(test_loader),
                    compute_rmse(train_loader), compute_rmse(test_loader)))

 epoch 0 | loss 0.9997 | Train Sat 0.000 | Test Sat 0.000 | Train RMSE 189.168 | Test RMSE 178.333 
 epoch 50 | loss 0.2877 | Train Sat 0.704 | Test Sat 0.688 | Train RMSE 8.566 | Test RMSE 9.309 
 epoch 100 | loss 0.2883 | Train Sat 0.710 | Test Sat 0.689 | Train RMSE 8.817 | Test RMSE 9.262 
 epoch 150 | loss 0.2822 | Train Sat 0.718 | Test Sat 0.701 | Train RMSE 9.077 | Test RMSE 8.714 
 epoch 200 | loss 0.2715 | Train Sat 0.718 | Test Sat 0.709 | Train RMSE 8.327 | Test RMSE 8.264 
 epoch 250 | loss 0.2811 | Train Sat 0.734 | Test Sat 0.708 | Train RMSE 8.626 | Test RMSE 8.222 
 epoch 300 | loss 0.2721 | Train Sat 0.728 | Test Sat 0.727 | Train RMSE 7.677 | Test RMSE 7.802 
 epoch 350 | loss 0.2567 | Train Sat 0.736 | Test Sat 0.730 | Train RMSE 7.728 | Test RMSE 7.726 
 epoch 400 | loss 0.2558 | Train Sat 0.717 | Test Sat 0.737 | Train RMSE 7.949 | Test RMSE 7.628 
 epoch 450 | loss 0.2684 | Train Sat 0.724 | Test Sat 0.730 | Train RMSE 7.645 | Test RMSE 7.703 


Notice that variables $x$ and $y$ are grounded batch by batch with new data arriving from the data loader. This is exactly what
we mean with $\mathcal{G}_{x \leftarrow \boldsymbol{B}}(\phi(x))$, where $B$ is a mini-batch sampled by the data loader.

Notice also that `SatAgg` is defined by one single axiom and returns the truth value corresponding to the evaluation
of the axiom.

Note that after 300 epochs the test RMSE is around 7, while at the beginning of the training it was around 62. This shows
the power of LTN in learning the regression task only using the satisfaction of a knowledge base as an objective.
